In [6]:
import mtt
from os.path import join, realpath
from os import getcwd
from pprint import pprint
sbfile = join(realpath(getcwd()),'kinase_cascade_stage-beta1.sb')
wiring = mtt.Wiring.fromFile2(sbfile, {}, 1.)
wiring.blocks[1].ratC = 1.
wiring.connect(wiring.blocks[1].rv_up, wiring.blocks[0].Cprod)
wiring

In [7]:
model = mtt.MTT(wiring)
model.draw(3)

# Digitized current config

In [8]:
for block in wiring.blocks:
    print(block.getDigitizedParameterString(model))

# Simulation

In [9]:
sbml = model.toSBML()
# print(sbml)
from roadrunner import RoadRunner
with open('out-sbml.xml', 'w') as f:
    f.write(sbml)
xmod = RoadRunner(sbml)
xmod.reset()
xmod.integrator.relative_tolerance = 1e-12
xmod.integrator.absolute_tolerance = 1e-20
# xmod.oneStep()
xmod.simulate(0,1000,1000,selections=['time']+xmod.getFloatingSpeciesIds())
xmod.plot()

values = {s:xmod[s] for s in xmod.getFloatingSpeciesIds()}
block_rates = [xmod['block{}_Add10'.format(k)] for k in (range(len(wiring.blocks)))]
# pprint(block_rates)
r_block_indices = model.getReactionBlockIndices()

# SBML Simulation

In [10]:
# SBML comparison
import tellurium as te
with open(sbfile) as f:
    rr_model = te.loada(f.read())

rr_model.reset()
rr_model.integrator.relative_tolerance = 1e-12
rr_model.integrator.absolute_tolerance = 1e-20
rr_model.selections = ['time','__s0','__s1','__s6','__s4','__s7','__s8','__s2','__s9','__s10','__s15']
rr_model.simulate(0,1000,2000)

for var,value in values.items():
    print('rel diff for {}: {:.3f}'.format(var,(rr_model[var]-value)/max(rr_model[var],1.)))
    rr_model[var] = value

# print('\nreactions:')
# for r_index in range(rr_model.getNumReactions()):
#     r_id = rr_model.getReactionIds()[r_index]
#     block_index = r_block_indices[r_index]
#     print('{} rel diff: {:.3f}'.format(r_id,(rr_model[r_id]-block_rates[block_index])/max(rr_model[r_id],1.)))
      
rr_model.plot()